In [20]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense,SimpleRNN, GRU
from tensorflow.keras.preprocessing.text import Tokenizer

In [1]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import nltk
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# from nltk.stem import SnowballStemmer as sno
# setofstopwords=set(stopwords.words('english'))
from sklearn.model_selection import train_test_split

# Load data

In [3]:
reviews=pd.read_csv('../data/amazon_reviews_us_Apparel_v1_00.tsv.gz.tsv',sep='\t')

In [4]:
reviews['label']=(reviews['verified_purchase']=='y').astype(int)

In [5]:
reviews.groupby('verified_purchase').count()

,star_rating,helpful_votes,total_votes,vine,review_headline,review_body,review_date,label
verified_purchase,,,,,,,,
n,593552,593552,593552,593552,593544,593491,593532,593552
y,5312781,5312781,5312781,5312781,5312734,5312053,5312680,5312781


In [7]:
samples=reviews[:5000]

In [8]:
samples.groupby('verified_purchase').count()

,star_rating,helpful_votes,total_votes,vine,review_headline,review_body,review_date,label
verified_purchase,,,,,,,,
n,1754,1754,1754,1754,1754,1754,1754,1754
y,3246,3246,3246,3246,3246,3245,3246,3246


In [9]:
undersample=samples[samples['label']==1].sample(1754)

In [10]:
undersample=pd.concat([undersample,samples[samples['label']==0]])
undersample = undersample.sample(frac=1,replace=False).reset_index(drop=True)

In [11]:
undersample.groupby('verified_purchase').count()

,star_rating,helpful_votes,total_votes,vine,review_headline,review_body,review_date,label
verified_purchase,,,,,,,,
n,1754,1754,1754,1754,1754,1754,1754,1754
y,1754,1754,1754,1754,1754,1754,1754,1754


In [12]:
selected_features=[ 'review_body','label']

In [13]:
# reviews['review_body']=reviews['review_body'].str.lower()

In [14]:
selected_reviews=undersample[selected_features]

In [15]:
selected_reviews['review_body']=selected_reviews['review_body'].str.replace(r'([^a-z\s]+)','')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
selected_reviews['review_body'].head()

0    these workout tights seem perfect so far they ...
1                                                 nice
2    this shirt had the strangest fit of anything i...
3    these wide leg yoga pants are super comfy ther...
4                        very lovely material must buy
Name: review_body, dtype: object

# Split data

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(selected_reviews['review_body'].values,\
                                                    selected_reviews['label'].values,\
                                                    test_size=0.30,\
                                                    shuffle=True)

In [18]:
tokenizer = Tokenizer(num_words= 50000)
tokenizer.fit_on_texts(X_train)

In [19]:
X_train_tok = tokenizer.texts_to_sequences(X_train)
X_test_tok = tokenizer.texts_to_sequences(X_test)

In [21]:
max_review_length = 600
X_train_pad = sequence.pad_sequences(X_train_tok, maxlen=max_review_length)
X_test_pad = sequence.pad_sequences(X_test_tok, maxlen=max_review_length)

# Compile models

In [22]:
embedding_vector_length = 32
top_words = 50000
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
# The output of SimpleRNN will be a 2D tensor of shape (batch_size, 128)
model.add(SimpleRNN(128))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 600, 32)           1600000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 128)               20608     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,620,737
Trainable params: 1,620,737
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(X_train_pad, Y_train, epochs=10, batch_size=64,validation_data=(X_test_pad,Y_test))
# Final evaluation of the model on test data
scores = model.evaluate(X_test_pad, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 2455 samples, validate on 1053 samples
Epoch 1/10
2455/2455 [==============================] - 13s 5ms/sample - loss: 0.6921 - accuracy: 0.5214 - val_loss: 0.6679 - val_accuracy: 0.5185
Epoch 2/10
2455/2455 [==============================] - 11s 4ms/sample - loss: 0.6522 - accuracy: 0.6151 - val_loss: 0.6381 - val_accuracy: 0.5252
Epoch 3/10
2455/2455 [==============================] - 11s 4ms/sample - loss: 0.5727 - accuracy: 0.7136 - val_loss: 0.5639 - val_accuracy: 0.6857
Epoch 4/10
2455/2455 [==============================] - 11s 4ms/sample - loss: 0.4163 - accuracy: 0.8289 - val_loss: 0.5226 - val_accuracy: 0.7312
Epoch 5/10
2455/2455 [==============================] - 11s 4ms/sample - loss: 0.8556 - accuracy: 0.6354 - val_loss: 0.7480 - val_accuracy: 0.4986
Epoch 6/10
2455/2455 [==============================] - 11s 5ms/sample - loss: 0.6143 - accuracy: 0.6692 - val_loss: 0.6447 - val_accuracy: 0.6505
Epoch 7/10
2455/2455 [==============================] - 11s 4ms/sample

In [24]:
top_words=50000
embedding_vector_length = 40
model = Sequential()
model.add(Embedding(top_words+1, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(16))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 600, 40)           2000040   
_________________________________________________________________
lstm (LSTM)                  (None, 600, 32)           9344      
_________________________________________________________________
lstm_1 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 2,012,537
Trainable params: 2,012,537
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.fit(X_train_pad, Y_train, epochs=10, batch_size=64,validation_data=(X_test_pad,Y_test))
# Final evaluation of the model on test data
scores = model.evaluate(X_test_pad, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 2455 samples, validate on 1053 samples
Epoch 1/10
2455/2455 [==============================] - 28s 11ms/sample - loss: 0.6436 - accuracy: 0.6391 - val_loss: 0.5419 - val_accuracy: 0.7217
Epoch 2/10
2455/2455 [==============================] - 23s 9ms/sample - loss: 0.5019 - accuracy: 0.7544 - val_loss: 0.5069 - val_accuracy: 0.7759
Epoch 3/10
2455/2455 [==============================] - 23s 9ms/sample - loss: 0.4095 - accuracy: 0.8191 - val_loss: 0.5188 - val_accuracy: 0.7825
Epoch 4/10
2455/2455 [==============================] - 23s 9ms/sample - loss: 0.3604 - accuracy: 0.8440 - val_loss: 0.4953 - val_accuracy: 0.7740
Epoch 5/10
2455/2455 [==============================] - 23s 9ms/sample - loss: 0.2879 - accuracy: 0.8823 - val_loss: 0.5009 - val_accuracy: 0.7778
Epoch 6/10
2455/2455 [==============================] - 23s 9ms/sample - loss: 0.2334 - accuracy: 0.9149 - val_loss: 0.5403 - val_accuracy: 0.7749
Epoch 7/10
2455/2455 [==============================] - 23s 9ms/sampl

In [28]:
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))

# The output of GRU will be a 3D tensor of shape (batch_size, timesteps, 256)
model.add(GRU(256, return_sequences=True))
model.add(SimpleRNN(128))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary() 

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 600, 40)           2000000   
_________________________________________________________________
gru_1 (GRU)                  (None, 600, 256)          228864    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 128)               49280     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 2,278,273
Trainable params: 2,278,273
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.fit(X_train_pad, Y_train, epochs=10, batch_size=64,validation_data=(X_test_pad,Y_test))
# Final evaluation of the model on test data
scores = model.evaluate(X_test_pad, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 2455 samples, validate on 1053 samples
Epoch 1/10
2455/2455 [==============================] - 82s 33ms/sample - loss: 0.6426 - accuracy: 0.6277 - val_loss: 0.5862 - val_accuracy: 0.7284
Epoch 2/10
2455/2455 [==============================] - 79s 32ms/sample - loss: 0.5189 - accuracy: 0.7352 - val_loss: 0.5092 - val_accuracy: 0.7398
Epoch 3/10
2455/2455 [==============================] - 78s 32ms/sample - loss: 0.4283 - accuracy: 0.8098 - val_loss: 0.7037 - val_accuracy: 0.7607
Epoch 4/10
2455/2455 [==============================] - 79s 32ms/sample - loss: 0.5112 - accuracy: 0.7401 - val_loss: 0.7197 - val_accuracy: 0.5005
Epoch 5/10
2455/2455 [==============================] - 78s 32ms/sample - loss: 0.7009 - accuracy: 0.4811 - val_loss: 0.6931 - val_accuracy: 0.5005
Epoch 6/10
2455/2455 [==============================] - 79s 32ms/sample - loss: 0.6933 - accuracy: 0.5132 - val_loss: 0.7014 - val_accuracy: 0.4995
Epoch 7/10
2455/2455 [==============================] - 78s 32ms

In [57]:
!python task_runner.py

2020-02-24 21:10:47.447407: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-02-24 21:10:47.452944: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2394450000 Hz
2020-02-24 21:10:47.453213: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x51c6fa0 executing computations on platform Host. Devices:
2020-02-24 21:10:47.453235: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
Train on 8 samples, validate on 4 samples
Epoch 1/10
8/8 [==============================] - 2s 206ms/sample - loss: 0.7159 - accuracy: 0.3750 - val_loss: 0.6964 - val_accuracy: 0.2500
Epoch 2/10
8/8 [==============================] - 0s 28ms/sample - loss: 0.6054 - accuracy: 1.0000 - val_loss: 0.6731 - val_accuracy: 0.7500
Epoch 3/10
8/8 [==============================] - 0s 28ms/sample - loss: 0.5360 - accuracy: 1.0000 - val_lo